# Домашнее задание 2. По чуть-чуть про pandas, sklearn, kNN

В этом задании вы в миниатюре пройдете цикл решения задачи анализа данных от предобработки данных до построения модели и оценки качества. Вы будете решать задачу кредитного скоринга (вернет клиент кредит или нет) на примере небольшой выборки данных. 

В процессе решения задания вы поближе познакомитесь с библиотеками pandas, sklearn и тем, как работают классы в python, а также немного вспомните numpy, самостоятельно реализовав 1NN (это очень просто, потому что сам алгоритм очень простой).

Все задачи нужно выполнять исключительно самостоятельно! Если что-то непонятно, задавайте вопросы в чате. При обнаружении списывания ноль за задание ставится и тому, кто списал, и тому, у кого списали.

Также во всем задании запрещено использование циклов (исключение - второе бонусное задание)! Все операции необходимо выполнять средствами numpy, pandas и sklearn.

Разделение выборки на обучение и контроль производится случайно (если только вы не указали конкретный random_state), поэтому результаты при разных запусках ноутбука могут получаться разные.

Считывание данных уже сделано:

In [2]:
import numpy as np
import pandas as pd

In [3]:
tab = pd.read_csv("https://raw.githubusercontent.com/nadiinchi/iad2019/master/materials/scoring.csv")

Выведите первые несколько строчек датасета, посмотрите названия колонок:

In [4]:
tab.head()

,account,duration,credit_history,purpose,amount,savings,employment,installment_rate,guarantors,residence,...,job,maintenance_people,telephone,foreign,real_estate,life_insurance,car,housing_rent,sex,target
0,0,18,4,furniture,3780,1,1,3,1,2,...,3,1,1,yes,0,0,1,0,male,1
1,1,24,2,car_new,1246,1,1,4,1,2,...,1,1,0,yes,1,0,0,0,male,0
2,0,15,2,car_used,3029,1,3,2,1,2,...,2,1,0,yes,0,0,1,0,male,1
3,0,36,3,television,4463,1,2,4,1,2,...,3,1,1,yes,0,0,1,0,male,0
4,-1,27,2,television,3416,1,2,3,1,2,...,3,1,0,yes,0,0,1,0,male,1


In [5]:
tab.columns

Index(['account', 'duration', 'credit_history', 'purpose', 'amount', 'savings',
       'employment', 'installment_rate', 'guarantors', 'residence', 'age',
       'credits_number', 'job', 'maintenance_people', 'telephone', 'foreign',
       'real_estate', 'life_insurance', 'car', 'housing_rent', 'sex',
       'target'],
      dtype='object')

Сначала несколько вопросов просто так:
    
 __[0.5 балла]__ Какой процент клиентов моложе 30 лет (колонка age) снимает жилье (колонка housing_rent)? А какой процент клиентов старше 40 лет снимает жилье?

In [63]:
np.mean(tab.loc[tab.age < 30, ['housing_rent']])

housing_rent    0.316017
dtype: float64

31,6% людей младше 30 лет снимают жилье

 __[0.5 балла]__ У скольких иностранцев (foreign == 'yes') число кредитов (credits_number) больше 2?

In [8]:
tab.loc[(tab.foreign == 'yes') & (tab.credits_number > 2)].shape


(19, 22)

в отсортированной таблице 19 строк, следовательно, 19 человек удовлетворяют заданному условию

 __[0.5 балла]__ Выведите колонки job, life_insurance и guarantors для клиентов с номерами 105, 158, 306 и 336 одной таблицей (воспользуйтесь индексацией):

In [65]:
tab[tab.index.isin([105, 158, 306, 336])][['job', 'life_insurance', 'guarantors']]


,job,life_insurance,guarantors
105,2,1,1
158,2,0,1
306,3,0,0
336,2,1,1


 __[0.5 балла]__ Выведите среднюю сумму кредита (колонка amount) в каждой категории (колонка purpose), воспользовавшись методом groupby:

In [11]:
tab.groupby('purpose').aggregate(np.mean)['amount']

purpose
business               4405.098361
car_new                3156.796178
car_used               5690.403509
domestic_appliances    1307.875000
education              3276.258065
furniture              3312.677966
others                 8681.700000
repairs                2792.769231
retraining             1156.000000
television             2554.237762
Name: amount, dtype: float64

Теперь по существу: нужно подготовить данные к построению модели.

 __[0.5 балла]__ Как вы видите, не все данные являются числовыми. С признаком purpose мы работать не будем - удалите его из датафрейма:

In [12]:
tab = tab[tab.columns.drop('purpose')]
tab.shape

(600, 21)

 __[0.5 балла]__ Необходимо перекодировать оставшиеся строковые значения в числа (закодировать пол female->0 и male->1 и колонку foreign числами yes->1 и no->0). Сделайте это с помощью pandas (без циклов, конечно же):

In [13]:
tab['encoded_sex'] = tab['sex'].map({'male':1, 'female':0})
tab = tab[tab.columns.drop('sex')]
tab.shape

(600, 21)

In [14]:
tab['is_foreign'] = tab['foreign'].map({'yes':1, 'no':0})
tab = tab[tab.columns.drop('foreign')]
tab.head()

,account,duration,credit_history,amount,savings,employment,installment_rate,guarantors,residence,age,...,job,maintenance_people,telephone,real_estate,life_insurance,car,housing_rent,target,encoded_sex,is_foreign
0,0,18,4,3780,1,1,3,1,2,35,...,3,1,1,0,0,1,0,1,1,1
1,1,24,2,1246,1,1,4,1,2,23,...,1,1,0,1,0,0,0,0,1,1
2,0,15,2,3029,1,3,2,1,2,33,...,2,1,0,0,0,1,0,1,1,1
3,0,36,3,4463,1,2,4,1,2,26,...,3,1,1,0,0,1,0,0,1,1
4,-1,27,2,3416,1,2,3,1,2,27,...,3,1,0,0,0,1,0,1,1,1


 __[0.5 балла]__ Целевая переменная хранится в колонке target (последняя колонка). Являются ли классы сбалансированными (одинаковое число 0 и 1)? Воспользуйтесь методом [value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html).

In [15]:
tab['target'].value_counts() 

1    300
0    300
Name: target, dtype: int64

да, явялются

Выведем статистику датасета:

In [17]:
tab.describe()

,account,duration,credit_history,amount,savings,employment,installment_rate,guarantors,residence,age,...,job,maintenance_people,telephone,real_estate,life_insurance,car,housing_rent,target,encoded_sex,is_foreign
count,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,...,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000
mean,0.083333,21.878333,2.410000,3470.491667,1.200000,2.348333,2.988333,0.901667,2.831667,35.181667,...,1.906667,1.150000,0.391667,0.268333,0.205000,0.343333,0.186667,0.500000,0.598333,0.965000
std,0.915306,12.749161,1.098747,3090.052187,0.924531,1.208069,1.131517,0.298013,1.103945,11.416182,...,0.662334,0.357369,0.488530,0.443462,0.404038,0.475218,0.389969,0.500417,0.490644,0.183933
min,-1.000000,4.000000,0.000000,250.000000,0.000000,0.000000,1.000000,0.000000,1.000000,19.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-1.000000,12.000000,2.000000,1363.250000,1.000000,2.000000,2.000000,1.000000,2.000000,26.000000,...,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,18.000000,2.000000,2347.000000,1.000000,2.000000,3.000000,1.000000,3.000000,33.000000,...,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,1.000000,1.000000
75%,1.000000,27.000000,4.000000,4318.750000,1.000000,3.000000,4.000000,1.000000,4.000000,41.000000,...,2.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000
max,2.000000,72.000000,4.000000,18424.000000,4.000000,4.000000,4.000000,1.000000,4.000000,75.000000,...,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


 __[0.5 балла]__ Проанализируйте таблицу. Есть ли признаки с пропущенными значениями? (ответ необходимо писать в этой же ячейке; к ответу сделайте пояснение)

нет, признаков с пропущенными значениями нет, так как количество значений во всех колонках равно 600. 

 __[0.5 балла]__ Запишите в переменную X матрицу объекты-признаки (выделите все колонки признаков, кроме target), а в переменную y - вектор правильных ответов (колонка target). К обоим объектам припишите .values, чтобы "достать" numpy-матрицу из датафрейма: далее мы везде будем работать с numpy-матрицами.

In [18]:
X = tab[tab.columns.drop('target')]
X.head()

,account,duration,credit_history,amount,savings,employment,installment_rate,guarantors,residence,age,credits_number,job,maintenance_people,telephone,real_estate,life_insurance,car,housing_rent,encoded_sex,is_foreign
0,0,18,4,3780,1,1,3,1,2,35,2,3,1,1,0,0,1,0,1,1
1,1,24,2,1246,1,1,4,1,2,23,1,1,1,0,1,0,0,0,1,1
2,0,15,2,3029,1,3,2,1,2,33,1,2,1,0,0,0,1,0,1,1
3,0,36,3,4463,1,2,4,1,2,26,2,3,1,1,0,0,1,0,1,1
4,-1,27,2,3416,1,2,3,1,2,27,1,3,1,0,0,0,1,0,1,1


In [19]:
Y = tab['target']

 __[0.5 балла]__ С помощью sklearn-функции train_test_split разделите данные на обучение и контроль (пропорцию оставьте по умолчанию). Ячейка ?train_test_split покажет документацию функции (с примерами использования), также можно загуглить документацию.

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
?train_test_split

In [22]:
X_tr, X_te, Y_tr, Y_te = train_test_split(X, Y)

Выведите форму (shape) обучающей и контрольной выборок:

In [62]:
print('Форма обучающей выборки:', X_tr.shape)
print('Форма тестовой выборки:', X_te.shape)

Форма обучающей выборки: (450, 20)
Форма тестовой выборки: (150, 20)


 __[0.5 балла]__ Как вы видели выше, все колонки имеют разный масштаб, это будет мешать нам при запуске метрического классификатора. Отнормируйте данные с помощью StandardScaler (интерфейс такой же, как мы изучали на семинаре):

In [24]:
from sklearn.preprocessing import StandardScaler

In [26]:
scaler = StandardScaler() 
scaler.fit(X_tr)
X_tr_trd = scaler.transform(X_tr)
X_te_trd = scaler.transform(X_te)

 __[0.5 балла]__ Теперь обучите классификатор 1-NN из sklearn на обучающей выборке, сделайте предсказания отдельно для обучающей и для контрольной выборки, на каждой выборке посчитайте точность (accuracy) с помощью accuracy_score. При создании объекта KNeighborsClassifier необходимо указать число соседей 1 (по умолчанию оно больше). 

Интерфейс KNeighborsClassifier такой же, как мы изучали на семинаре, а accuracy-score принимает на вход вектор предсказаний и вектор правильных ответов.

In [27]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [28]:
?KNeighborsClassifier

In [29]:
model = KNeighborsClassifier(n_neighbors = 1)
model.fit(X_tr_trd, Y_tr)
preds_tr = model.predict(X_tr_trd)
preds_te = model.predict(X_te_trd)

In [30]:
print('Train: ', accuracy_score(preds_tr, Y_tr), '\n', 'Test: ', accuracy_score(preds_te, Y_te), sep='')

Train: 1.0
Test: 0.64


 __[4 балла,  включая my_accuracy и тестирование реализации]__ Наконец, реализуйте свой 1NN, воспользовавшись шаблоном ниже. Ваш класс должен уметь работать в двух режимах: в евклидовой метрикой и с манхэттанской метрикой (формулы метрик ниже). Напоминаем принцип работы 1NN: чтобы сделать предсказание для объекта, он выбирает ближайший объект из обучения и возвращает такой же класс, как у него.
* в методе \__init\__ необходимо запомнить, с какой метрикой мы работаем
* в методе fit нужно запомнить выборку - больше при обучении kNN ничего делать не нужно
* в методе predict нужно сделать три шага:
    * сначала построить матрицу расстояний между контрольными и обучающими объектами. Для этого можно использовать функцию [cdist](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html) (прочитайте ее документацию).
    * затем взять argmin по строкам - получится вектор индексов ближайших обучающих объектов для каждого контрольного объекта
    * выбрать компоненты y_train, отвечающие компонентам вектора индексов. Иными словами, вернуть те же метки классов, что у ближайших обучающих объектов.
    
Формулы метрик (расстояний):
* евклидова: $$\rho(x, y) = \sqrt{\sum_{i=1}^d (x_i - y_i)^2}$$
* манхэттанская: $$\rho(x, y) = \sum_{i=1}^d |x_i - y_i|$$

Модуль в numpy вычисляется как np.abs.

In [31]:
from scipy.spatial.distance import cdist

In [33]:
class MyKNN:
    def __init__(self, metric):
        if metric == 'euclid':
            self.metric = 'euclid'
        elif metric == 'manhattan':
            self.metric = 'manhattan'
        
    def fit(self, X_tr, y_tr):
        self.sample = X_tr
        self.target = y_tr
        
    def predict(self, X_te):
        if self.metric == 'euclid':
            self.distance = cdist(self.sample, X_te,'euclidean')
            self.indx = np.argmin(self.distance, axis = 0)
            self.y_pred = self.target.iloc[self.indx]
            return self.y_pred
        
        elif self.metric == 'manhattan':
            self.distance = cdist(self.sample, X_te,'minkowski', p = 1)
            self.indx = np.argmin(self.distance, axis = 0)
            self.y_pred = self.target.iloc[self.indx]
            return self.y_pred

Реализуйте также функцию вычисления точности - доли правильных ответов:

In [51]:
def my_accuracy(y_pred, y_test):
    return np.mean(y_pred.reset_index(drop = True) == y_test.reset_index(drop = True))

Пришло время запустить ваш классификатор и вычислить точность с помощью вашей функции. Обучите классификатор, сделайте предсказания для обучающей и контрольной выборок и посчитайте точность в обоих случаях. Сделайте это два раза - ддя евклидовой и манхэттанской метрик. Для евклидовой метрики цифры должны получиться точно такие же, как для реализации sklearn.

In [59]:
my_model = MyKNN('euclid')
my_model.fit(X_tr_trd, Y_tr)
my_preds_tr_euc = my_model.predict(X_tr_trd)
my_preds_te_euc = my_model.predict(X_te_trd)
print('Euclidean:', '\n', 'Train: ', my_accuracy(my_preds_tr_euc, Y_tr), '\n', 'Test: ', my_accuracy(my_preds_te_euc, Y_te), sep='')

Euclidean:
Train: 1.0
Test: 0.64


In [60]:
my_model = MyKNN('manhattan')
my_model.fit(X_tr_trd, Y_tr)
my_preds_tr_man = my_model.predict(X_tr_trd)
my_preds_te_man = my_model.predict(X_te_trd)
print('Manhattan', '\n', 'Train: ', my_accuracy(my_preds_tr_man, Y_tr), '\n', 'Test: ', my_accuracy(my_preds_te_man, Y_te), sep='')

Manhattan
Train: 1.0
Test: 0.6666666666666666


### Бонусное задание: реализация kNN
Добавьте в ваш KNN параметр k (число соседей) и реализуйте k-NN, который возвращает самый популярный класс среди k ближайших соседей объекта. Если число ближайших объектов для разных классов совпадает, возвращайте класс с меньшим номером. За реализацию, работающую для любого числа классов - 3 балла (посмотрите функции np.bincount, np.apply_along_axis (к сожалению, bincount не применяется построчно), np.argpartition), за реализацию, работающую только для двух классов - 2 балла. Не забудьте протестировать свой алгоритм :) В этом задании циклы использовать нельзя!
